In [1]:
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 
import re
import scipy as sp

plt.style.use('fivethirtyeight')

# 노트북 안에 그래프 그리기
%matplotlib inline
# 그래프 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일 사용
plt.style.use('ggplot')
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus']=False

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

# 한글 font 설정
import matplotlib.font_manager as fm

font_name = fm.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
plt.rc('font',family = font_name)
mpl.rcParams['axes.unicode_minus'] = False

# 데이터 로드

In [2]:
final_data = pd.read_csv("data/final_data.csv")

In [3]:
final_data.info()
final_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6438227 entries, 0 to 6438226
Data columns (total 29 columns):
CLNT_ID                 int64
SESS_ID                 int64
HITS_SEQ                int64
PD_C                    int64
PD_ADD_NM               object
PD_BRA_NM               object
PD_BUY_AM               int64
PD_BUY_CT               int64
PD_NM                   object
CLAC1_NM                object
CLAC2_NM                object
CLAC3_NM                object
CLNT_GENDER             object
CLNT_AGE                float64
MEMBER                  int64
SESS_SEQ                int64
SESS_DT                 object
TOT_PAG_VIEW_CT         int64
TOT_SESS_HR_V           int64
DVC_CTG_NM              object
ZON_NM                  object
CITY_NM                 object
SESS_MONTH              int64
SESS_DAY                int64
SESS_WEEK               int64
KWD_NM                  object
SEARCH_CNT              float64
search_purpose          int64
search_purpose_index    float64

CLNT_ID                 0
SESS_ID                 0
HITS_SEQ                0
PD_C                    0
PD_ADD_NM               1
PD_BRA_NM               0
PD_BUY_AM               0
PD_BUY_CT               0
PD_NM                   0
CLAC1_NM                0
CLAC2_NM                0
CLAC3_NM                0
CLNT_GENDER             0
CLNT_AGE                0
MEMBER                  0
SESS_SEQ                0
SESS_DT                 0
TOT_PAG_VIEW_CT         0
TOT_SESS_HR_V           0
DVC_CTG_NM              0
ZON_NM                  0
CITY_NM                 0
SESS_MONTH              0
SESS_DAY                0
SESS_WEEK               0
KWD_NM                  0
SEARCH_CNT              0
search_purpose          0
search_purpose_index    0
dtype: int64

# 선호지수 생성

In [4]:
test = final_data[final_data["search_purpose"]==1]
test.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,SESS_MONTH,SESS_DAY,SESS_WEEK,KWD_NM,SEARCH_CNT,search_purpose,search_purpose_index
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다 스무드,1.0,1,0.285714
3,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다,23.0,1,1.000000
5,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,이솝,3.0,1,2.500000
10,4142395,6158159,43,176911,notset,아베다,90000,1,6월스무드인퓨전샴푸리터세트,퍼스널케어,헤어케어,린스/컨디셔너,F,40.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다 스무드,1.0,1,0.571429
11,4142395,6158159,43,176911,notset,아베다,90000,1,6월스무드인퓨전샴푸리터세트,퍼스널케어,헤어케어,린스/컨디셔너,F,40.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다,23.0,1,1.000000


In [5]:
test1 = test.groupby(["CLNT_ID", "SESS_ID"]).mean().reset_index()
test1 = test1.groupby(["PD_C"]).mean().reset_index()

X = test1[["TOT_SESS_HR_V", "TOT_PAG_VIEW_CT", "HITS_SEQ"]]
X

from sklearn.preprocessing import StandardScaler
x_std = StandardScaler().fit_transform(X)
x_std

features = x_std.T
features

covariance_matrix = np.cov(features)
print(covariance_matrix)

[[1.00000182 0.60330317 0.52926506]
 [0.60330317 1.00000182 0.8566398 ]
 [0.52926506 0.8566398  1.00000182]]


In [6]:
eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)
eig_vals
eig_vecs

eig_vals[0] / sum(eig_vals)

0.7789466995781604

In [7]:
projected_X = x_std.dot(eig_vecs.T[0])
projected_X

array([-1.98138203, -0.61016643, -1.65341878, ...,  1.38438265,
        0.29195696, -0.38312777])

In [23]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_projected_X = min_max_scaler.fit_transform(projected_X.reshape(-1, 1))

test1["like_index"] = min_max_projected_X.flatten()
test1.head()

,PD_C,CLNT_ID,SESS_ID,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,CLNT_AGE,MEMBER,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,SESS_MONTH,SESS_DAY,SESS_WEEK,SEARCH_CNT,search_purpose,search_purpose_index,like_index
0,2.0,1.373088e+06,9368697.00,26.000000,39000.000000,1.0,20.000000,1.000000,43.000000,25.000000,106.000000,4.000000,27.000000,4.000000,1.00000,1.0,1.375000,0.022968
1,7.0,3.012601e+06,4963116.40,90.405263,26268.421053,1.0,34.105263,0.936842,218.473684,114.621053,1629.073684,6.852632,13.147368,2.431579,2.10807,1.0,1.064750,0.123894
2,10.0,5.854950e+05,10168362.00,34.000000,9900.000000,1.0,40.000000,1.000000,210.000000,45.000000,663.000000,4.000000,14.000000,5.000000,2.50000,1.0,2.000000,0.047107
3,14.0,1.542686e+06,6345906.00,94.000000,83000.000000,1.0,40.000000,1.000000,215.000000,98.000000,1158.000000,6.000000,16.000000,5.000000,1.00000,1.0,0.666667,0.108555
4,17.0,2.510499e+06,3113107.75,58.375000,27900.000000,1.0,38.750000,0.750000,251.750000,90.875000,1205.000000,7.750000,16.500000,1.125000,1.56250,1.0,1.522917,0.089556


In [24]:
test2 = test1[["PD_C", "like_index"]]

In [25]:
a = pd.merge(final_data, test2, on="PD_C", how="inner")
a.isnull().sum()

CLNT_ID                 0
SESS_ID                 0
HITS_SEQ                0
PD_C                    0
PD_ADD_NM               1
PD_BRA_NM               0
PD_BUY_AM               0
PD_BUY_CT               0
PD_NM                   0
CLAC1_NM                0
CLAC2_NM                0
CLAC3_NM                0
CLNT_GENDER             0
CLNT_AGE                0
MEMBER                  0
SESS_SEQ                0
SESS_DT                 0
TOT_PAG_VIEW_CT         0
TOT_SESS_HR_V           0
DVC_CTG_NM              0
ZON_NM                  0
CITY_NM                 0
SESS_MONTH              0
SESS_DAY                0
SESS_WEEK               0
KWD_NM                  0
SEARCH_CNT              0
search_purpose          0
search_purpose_index    0
like_index              0
dtype: int64

# 영화추천 기반 상품 소분류 추천

    - CLAC3_NM 을 검색하면 비슷한 상관관계를 갖는 소분류 중 가장 높은 선호지수를 갖는 소분류 추출
    
    - 소분류에 따른 SESSION_ID의 선호지수(like_index)로 구현
    
    - 적합한 선호지수 개발시 신뢰성 있는 정보 나올 것으로 추정
    
    - 영화 장르 대신 상품명을 통해 상품도 함께 출력될 수 있도록 가능?
        : 상품명에 대한 자연어 처리 필요(PD_C와 연결되는 고유값 갖도록)

In [26]:
meta = a.groupby("CLAC3_NM").mean().reset_index()
meta

,CLAC3_NM,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_BUY_AM,PD_BUY_CT,CLNT_AGE,MEMBER,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,SESS_MONTH,SESS_DAY,SESS_WEEK,SEARCH_CNT,search_purpose,search_purpose_index,like_index
0,2단우산,3.558813e+06,7.303852e+06,116.821429,750279.950893,1.161830e+04,1.285714,38.616071,0.723214,274.955357,179.383929,3499.080357,5.580357,13.183036,2.602679,1.812500,0.450893,0.736816,0.127257
1,3단우산,3.497582e+06,6.421611e+06,139.933054,668727.365212,1.335264e+04,1.056186,37.005380,0.752540,242.323969,166.291692,3094.602510,6.006575,14.870293,2.739988,2.033473,0.541542,0.813382,0.117461
2,BB/파운데이션/컴팩트류,3.337656e+06,5.637697e+06,115.752271,591959.420962,5.185081e+04,1.072909,31.187415,0.796466,154.103272,145.890005,2527.083068,6.431445,14.875111,2.859867,1.762555,0.463433,0.321066,0.097070
3,DIY가구,3.162695e+06,5.876957e+06,75.114286,738139.685714,1.901337e+06,14.342857,37.714286,0.885714,31.542857,93.657143,1775.828571,6.314286,14.571429,2.028571,2.028571,0.857143,0.400000,0.109750
4,DSLR,3.365455e+06,6.673669e+06,68.090476,591636.254762,6.883133e+05,1.004762,30.523810,0.245238,106.997619,100.728571,2273.397619,5.940476,12.626190,2.859524,2.188095,0.557143,0.268290,0.069365
5,LCD,2.950620e+06,3.018153e+06,77.941176,650189.000000,3.745871e+05,1.000000,35.882353,0.647059,103.058824,103.529412,2186.588235,8.058824,8.411765,3.823529,1.823529,0.294118,0.215686,0.148017
6,LED,3.111279e+06,5.766863e+06,95.937817,589593.815990,5.119793e+05,1.057107,37.842640,0.657360,185.519036,148.092640,3526.148477,6.338832,15.447970,2.950508,1.623096,0.411168,0.270149,0.110451
7,OLED,2.681105e+06,5.284087e+06,64.000000,276208.076923,3.040100e+06,1.000000,24.615385,0.307692,21.692308,112.538462,2924.307692,6.846154,8.076923,5.230769,2.769231,0.461538,0.529604,0.084101
8,PC부품,3.457353e+06,5.384793e+06,106.398049,581922.639024,1.704191e+04,1.199350,36.858537,0.744390,269.192195,135.268293,2650.318374,6.539187,15.755447,2.681951,1.753496,0.463740,0.334843,0.096966
9,UHD,3.574488e+06,5.164492e+06,89.857143,328287.201970,1.020321e+06,1.000000,33.842365,0.704433,84.610837,140.305419,2756.507389,6.669951,15.463054,3.059113,1.950739,0.512315,0.332940,0.128325


In [27]:
meta = meta[["CLAC3_NM"]]
meta

,CLAC3_NM
0,2단우산
1,3단우산
2,BB/파운데이션/컴팩트류
3,DIY가구
4,DSLR
5,LCD
6,LED
7,OLED
8,PC부품
9,UHD


In [28]:
ratings = a.groupby(["SESS_ID", "CLAC3_NM"]).mean().reset_index()
ratings

,SESS_ID,CLAC3_NM,CLNT_ID,HITS_SEQ,PD_C,PD_BUY_AM,PD_BUY_CT,CLNT_AGE,MEMBER,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,SESS_MONTH,SESS_DAY,SESS_WEEK,SEARCH_CNT,search_purpose,search_purpose_index,like_index
0,76,영화/문화모바일상품권,5300081.0,6.0,568749.0,8000.0,10.0,30.0,1.0,39.0,9.0,97.0,9.0,30.0,6.0,3.000000,1.000000,1.000000,0.020786
1,237,페이셜클렌저,1515925.0,47.0,418090.0,36000.0,1.0,30.0,1.0,34.0,41.0,239.0,9.0,30.0,6.0,2.000000,1.000000,0.714286,0.122475
2,279,블러셔/쉐이딩/하이라이터,6347849.0,44.0,158791.0,85000.0,1.0,40.0,1.0,45.0,38.0,298.0,9.0,30.0,6.0,1.000000,1.000000,0.800000,0.142449
3,322,아이라이너,196114.0,71.0,746059.5,28000.0,1.0,30.0,1.0,2.0,57.0,347.0,9.0,30.0,6.0,1.000000,1.000000,0.714286,0.108095
4,382,여성등산티셔츠/탑,6157572.0,89.0,128655.0,59400.0,1.0,40.0,1.0,68.0,72.0,399.0,9.0,30.0,6.0,1.500000,1.000000,0.888889,0.099548
5,390,유아용화장품,153619.0,36.0,3158.0,65000.0,1.0,20.0,1.0,126.0,72.0,445.0,9.0,30.0,6.0,1.000000,0.500000,0.400000,0.107233
6,394,페이셜클렌저,6415640.0,50.0,631487.0,32500.0,1.0,40.0,1.0,612.0,94.0,446.0,9.0,30.0,6.0,1.000000,1.000000,1.000000,0.088413
7,406,미스트,2364862.0,71.0,644924.0,15000.0,1.0,20.0,1.0,18.0,55.0,227.0,9.0,30.0,6.0,1.000000,1.000000,1.666667,0.056268
8,436,여성티셔츠/탑,6355714.0,54.0,451469.0,12330.0,1.0,20.0,1.0,24.0,77.0,422.0,9.0,30.0,6.0,1.000000,1.000000,1.333333,0.064853
9,438,고양이사료,6573973.0,36.0,9617.0,45900.0,1.0,40.0,0.0,133.0,32.0,321.0,9.0,30.0,6.0,1.000000,1.000000,1.285714,0.086720


In [29]:
ratings = ratings[["SESS_ID", "CLAC3_NM", "like_index"]]
ratings.head()

,SESS_ID,CLAC3_NM,like_index
0,76,영화/문화모바일상품권,0.020786
1,237,페이셜클렌저,0.122475
2,279,블러셔/쉐이딩/하이라이터,0.142449
3,322,아이라이너,0.108095
4,382,여성등산티셔츠/탑,0.099548


In [30]:
ratings.describe()

,SESS_ID,like_index
count,1.527466e+06,1.527466e+06
mean,5.785317e+06,1.265596e-01
std,3.062621e+06,6.796503e-02
min,7.600000e+01,1.414926e-03
25%,3.258001e+06,8.473334e-02
50%,5.963994e+06,1.098247e-01
75%,8.366485e+06,1.494639e-01
max,1.097794e+07,8.810655e-01


In [31]:
data = pd.merge(meta, ratings, on="CLAC3_NM", how="inner")
data.head()

,CLAC3_NM,SESS_ID,like_index
0,2단우산,1565608,0.159619
1,2단우산,1588301,0.124407
2,2단우산,1601846,0.108611
3,2단우산,2058829,0.068325
4,2단우산,2064666,0.080355


In [32]:
matrix = data.pivot_table(index="SESS_ID", columns="CLAC3_NM", values="like_index")
matrix

CLAC3_NM,2단우산,3단우산,BB/파운데이션/컴팩트류,DIY가구,DSLR,LCD,LED,OLED,PC부품,UHD,가발/부분가발,가습기,각티슈/미용티슈,감,거들,거실수예소품,거실화/실내화,건강보조식품세트,건조기,계량도구,고데기,고무장갑,고양이간식,고양이건강용품,고양이모래/배변용품,고양이목욕/위생용품,고양이사료,고양이식기/급수,고양이의류/악세서리,고양이이동장,고양이장난감,고양이캣타워/실내용품,골프가방,골프공,골프연습장비,골프장갑,골프채,골프채세트,골프패션잡화,골프필드용품,골프화,공간박스,공기청정기,공병/모델링팩전용도구,공유기,과실주병,과일즙,교자상/다용도상,구강청정제,구명조끼/안전용품,국그릇,국자/뒤지개/주걱,군모,귀걸이,귤류,그늘막/타프,그릴/구이불판,글루코사민,기름종이,기저귀크림/파우더,기차/레일완구,기타견과류,기타광학기기,기타구강관리용품,기타국산과일류,기타기능성음료,기타남성양말류,기타냉동간편식,기타냉방가전,기타냉장고,기타등산용품,기타모바일기기,기타모바일액세서리,기타모자,기타물놀이용품,기타배낭소품,기타보석류,기타에어컨,기타여성속옷,기타여성양말류,기타여성의류아우터,기타여행용가방,기타영양제,기타영유아아우터,기타요가/필라테스소품,기타유아동양말류,기타유아동화,기타유아안전용품,기타이미용가전,기타일반문구/사무용품,기타자동차가전기기,기타정리용품,기타조리도구,기타주방가전,기타주방정리용품/소모품,기타청소기,기타카메라,기타캠핑용품,기타컴퓨터액세서리,기타패션잡화,기타피트니스기구,김치류,남녀공용향수,남녀공용향수세트,남성가디건,남성골프남방셔츠,남성골프니트/가디건,남성골프바지,남성골프베스트,남성골프스웨트셔츠/후드/집업,남성골프점퍼/재킷,남성골프티셔츠,남성골프패딩,남성남방셔츠,남성내의,남성덧신류,남성등산/아웃도어세트,남성등산바지,남성등산베스트,남성등산전신,남성등산점퍼/재킷,남성등산티셔츠,남성등산패딩,남성런닝/트레이닝화,남성런닝셔츠,남성로퍼,남성머니클립,남성발가락양말,남성백팩,남성베스트,남성부츠,남성비치웨어,남성사파리,남성샌들,남성서류가방,남성선글라스,남성속옷세트,남성숄더/크로스백,남성수면양말,남성수영복,남성스니커즈,남성스웨터/풀오버,남성스웨트셔츠/후드/집업,남성스킨케어세트,남성스포츠베스트,남성스포츠샌들/슬리퍼,남성스포츠속옷,남성스포츠스웨트셔츠/후드/집업,남성스포츠점퍼/재킷,남성스포츠티셔츠,남성스포츠패딩,남성스포츠화부속품,남성슬리퍼,남성슬립온,남성시계,남성신발부속품,남성실내복,남성양말선물세트,남성용선크림/메이크업류,남성용스킨케어류,남성용클렌저,남성일반스포츠바지,남성일반양말,남성일반지갑,남성잠옷,남성점퍼,남성정장바지,남성정장세트,남성정장셔츠,남성정장재킷,남성정장화,남성청바지,남성카드/명함지갑,남성캐주얼바지,남성캐주얼재킷,남성캐쥬얼스포츠양말,남성컴포트화,남성코트,남성클러치백,남성트레이닝복,남성트렌치코트,남성티셔츠,남성패딩,남성팬티,남성향수,남성힙색,남아가디건,남아레깅스,남아바지,남아베스트,남아셔츠,남아스웨터/풀오버,남아스웨트셔츠/후드/집업,남아실내복,남아의류세트,남아잠옷,남아청바지,남아티셔츠/탑,냄비,냅킨,냉동고,냉동국탕류,냉동떡볶이,냉동만두,냉동면,냉동밥,냉동부침,냉동튀김,냉동피자,냉동핫도그,냉온풍기,네비게이션,네일세트,네일케어도구,넥워머,노트북,노트북가방,놀이방매트,니삭스/오버니삭스,다기류,다이어트보조식품,단무지,닭가슴살,대접/볼,데스크탑/올인원PC,데오도란트,도마,도시락/찬합,독서대,돼지고기선물세트,드럼세탁기,등산지팡이/스틱,등산화,디저트포크/스푼,딸기,땅콩,뚜껑형김치냉장고,뚝배기,러닝/워킹머신,레고,레저모바일상품권,로만쉐이드/벌룬쉐이드,로봇청소기,롤스크린,롤티슈,롤플레잉완구,루테인,리모컨/액세서리,린스/컨디셔너,립글로즈/틴트,립밤/립스크럽,립스틱/립라이너,마스카라,마우스,마카다미아,만년필,매실,매직/보드마카,매트리스,머플러,멀티형에어컨,메론,메이크업베이스/프라이머,메이크업브러쉬,메이크업세트,메탈미용소도구,면봉/화장솜,명함정리용품,모니터,모바일배터리/충전기,모빌,모유보관용품,목걸이,목욕가운,목욕용장난감,목욕타올,무릎담요,무선조종,무화과,문구세트,물걸레청소기,물병,물안경,물티슈,미니자동차,미러리스,미술/창작완구,미스트,미용보조식품,미용비누,바구니,바디보습,바디슬리밍/리프팅,바디워시,바디케어세트,바란스,바운서/쏘서/보행기,반상기세트/홈세트,반죽기/제면기,반지,반찬세트,반찬통/밀폐용기,발,발찌,발포비타민,발효원액,밤,밥공기,방석/방석커버,방울토마토,배,배낭,배냇저고리,벙거지,벽걸이형에어컨,변기시트/커버,보드게임,보드류,보석세트,보온도시락,보온병/텀블러,복근/벨트마사지기구,복숭아,볶음탕용닭,볼펜,봉제인형,부분세탁제,부채,분말세탁세제,분말표백제,붕붕카/스프링카/흔들말,붙박이장,브래지어,브로치,블라인드/버티컬,블랙박스,블랜더,블러셔/쉐이딩/하이라이터,블록,비니,비닐장갑,비타민/에너지음료,사과,사무용/학생용가구세트,사인펜,삼계탕용닭,생리대,생수,생활모바일상품권,샤워/목욕도구/목욕헤어밴드,샤워커튼,샤프/샤프심,샴푸,샴푸/린스세트,서랍장/수납장,서류정리용품,석류,선반/걸이,선반장/행거,선크림류,선풍기,섬유유연제/향기지속제,성인담요,성인매트리스커버,성인베개/베개커버,성인요/요커버,성인이불/이불커버,성인침구세트,성인침구속통/솜,성인침대커버/스커트,성인패드/스프레드,세탁비누,소파,소품가방,소프트웨어,속눈썹/쌍꺼풀,손수건,손싸개/발싸개,솥,수건,수박,수세미/솔,수영가방,수영모자,수예소품속통/솜,수유패드/보조용품,수정용품,숙취해소음료,순금/순은/장식품,숟가락/젓가락,스냅백,스카프,스케이트보드/킥보드,스킨/토너,스킨케어디바이스,스킨케어세트,스타킹,스탠드형김치냉장고,스탠드형에어컨,스테이플러,스텝퍼/트위스트,스툴/리빙의자,스팀청소기,스포츠가방,스포츠두건/머플러/마스크,스포츠모자,스포츠목걸이/팔찌,스포츠선글라스,스포츠시계,스포츠아대/헤어밴드,스포츠양말,스포츠음료,스폰지/퍼프,스피커,슬립,승마운동기,시계세트,시공가구,식기건조기,식기건조대/수저통,식기세척기,식음료모바일상품권,식탁,식탁세트,식탁의자,신발장,싱크대,싱크대/배수구용품,싸인물/자석/압핀,썬캡,아기띠/캐리어,아기체육관/러닝홈,아동모,아동비치웨어,아동수영복,아동용가방,아동우산,아몬드,아이라이너,아이브로우,아이섀도우,아이젠,아이케어,아쿠아슈즈,안경소품,안경테,압력솥,애견간식,애견건강용품,애견목욕/위생용품,애견사료,애견식기/물병,애견의류/악세서리,애견이동장,애견장난감/훈련,애견주거/실내용품,액상세탁세제,액상표백제,야구모자,야외용돗자리,양념통,양문형냉장고,양산,어린이홍삼,얼음/빙수용품,에멀젼/로션,에센스/세럼,에어로빅복,에어워셔,여성가디건,여성가방액세서리,여성가운,여성골프남방셔츠,여성골프니트/가디건,여성골프바지,여성골프베스트,여성골프스웨트셔츠/후드/집업,여성골프스커트,여성골프전신/원피스,여성골프점퍼/재킷,여성골프티셔츠/탑,여성골프패딩,여성남방셔츠,여성내의,여성덧신류,여성등산/아웃도어세트,여성등산바지,여성등산베스트,여성등산전신/원피스,여성등산점퍼/재킷,여성등산티셔츠/탑,여성등산패딩,여성런닝/트레이닝화,여성런닝셔츠/캐미솔,여성레깅스,여성레인부츠/슈즈,여성로퍼,여성바지,여성발가락양말,여성방한화,여성백팩,여성베스트,여성부츠,여성블라우스,여성비치웨어,여성사파리,여성샌들,여성선글라스,여성세정제,여성속옷세트,여성숄더백,여성수면양말,여성수영복,여성스니커즈,여성스웨터/풀오버,여성스웨트셔츠/후드/집업,여성스커트,여성스포츠베스트,여성스포츠샌들/슬리퍼,여성스포츠속옷,여성스포츠스웨트셔츠/후드/집업,여성스포츠스커트,여성스포츠전신/원피스,여성스포츠점퍼/재킷,여성스포츠티셔츠/탑,여성스포츠패딩,여성슬리퍼,여성슬립온,여성슬링백,여성시계,여성신발부속품,여성실내복,여성양말선물세트,여성오픈토,여성원피스,여성일반스포츠바지,여성일반양말,여성일반지갑,여성임부속옷,여성잠옷,여성재킷,여성점퍼,여성점프수트/오버롤,여성쪼리,여성청바지,여성카드/명함지갑,여성캐쥬얼스포츠양말,여성컴포트화,여성코트,여성크로스백,여성클러치백,여성타이즈,여성토트백,여성트레이닝복,여성트렌치코트,여성티셔츠/탑,여성패딩,여성팬티,여성펌프스,여성플랫,여성향수,여성향수세트,여아가디건,여아남방셔츠,여아레깅스,여아바지,여아베스트,여아블라우스,여아스웨터/풀오버,여아스웨트셔츠/후드/집업,여아스커트,여아실내복,여아잠옷,여아재킷,여아점퍼,여아청바지,여아코트,여아티셔츠/탑,여아패딩,여행용세트,여행용소품,역할놀이,연필,연필깎이

In [33]:
def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c**2) * np.sum(s2_c**2))

def recommend(input_CLAC3_NM, matrix, n):
    result = []
    for title in matrix.columns:
        if title == input_CLAC3_NM:
            continue
            
        cor = pearsonR(matrix[input_CLAC3_NM], matrix[title])
        
        if np.isnan(cor):
            continue
        else:
            result.append((title, "{:.2f}".format(cor)))
            
    result.sort(key=lambda r:r[1], reverse=True)
    
    return result[:n]

In [34]:
### 모델 테스트

recommend_result = recommend("PC부품", matrix, 5)
pd.DataFrame(recommend_result, columns=["Title", "Correlation"])

KeyboardInterrupt: 